In [7]:
import pandas as pd
full_data = pd.read_csv('BRaTSdataset_filenames.csv')
test_data = pd.read_csv('test_file_paths_clean.csv')
# print number of rows in each dataframe
print(full_data.shape)
print("Rows in full data: ",len(full_data))
print("Rows in test data: ",len(test_data))


# Remove test_data rows from full_data to avoid overlap
train_data = full_data[~full_data['path'].isin(test_data['path'])]
train_data = train_data.reset_index(drop=True)


print("Rows in full data: ",len(train_data))
print("Rows in test data: ",len(test_data))

# print rows that appear in both dataframes
print("Rows in both dataframes: ",len(train_data[train_data['path'].isin(test_data['path'])]))

(484, 1)
Rows in full data:  484
Rows in test data:  49
Rows in full data:  435
Rows in test data:  49
Rows in both dataframes:  0


In [5]:
import nibabel as nib
import tensorflow as tf
from tensorflow import keras as K

class Metrics:
    def precision_coef(self, target, prediction, axis=(1, 2), smooth=0.0001):
        """
        Precision
        \frac{\left | T \right | \cap \left | P \right |}{\left | P \right |}
        where T is the ground truth mask and P is the prediction mask
        """
        prediction = K.backend.round(prediction)  # Round to 0 or 1

        true_positives = tf.reduce_sum(target * prediction, axis=axis)
        predicted_positives = tf.reduce_sum(prediction, axis=axis)
        numerator = true_positives + smooth
        denominator = predicted_positives + smooth
        coef = numerator / denominator

        return tf.reduce_mean(coef)

    def dice_coef(self, target, prediction, axis=(1, 2), smooth=0.0001):
        """
        Sorenson Dice
        \frac{  2 \times \left | T \right | \cap \left | P \right |}{ \left | T \right | +  \left | P \right |  }
        where T is ground truth mask and P is the prediction mask
        """
        prediction = K.backend.round(prediction)  # Round to 0 or 1

        intersection = tf.reduce_sum(target * prediction, axis=axis)
        union = tf.reduce_sum(target + prediction, axis=axis)
        numerator = tf.constant(2.) * intersection + smooth
        denominator = union + smooth
        coef = numerator / denominator

        return tf.reduce_mean(coef)

    def iou_coef(self, target, prediction, axis=(1, 2), smooth=0.0001):
        """
        Intersection over Union (IoU)
        \frac{\text{Intersection}}{\text{Union}} = \frac{\text{TP}}{\text{TP} + \text{FP} + \text{FN}}
        where T is the ground truth mask and P is the prediction mask
        """
        prediction = K.backend.round(prediction)  # Round to 0 or 1

        intersection = tf.reduce_sum(target * prediction, axis=axis)
        union = tf.reduce_sum(target + prediction, axis=axis) - intersection
        coef = (intersection + smooth) / (union + smooth)

        return tf.reduce_mean(coef)

    def soft_dice_coef(self, target, prediction, axis=(1, 2), smooth=0.0001):
        """
        Sorenson (Soft) Dice  - Don't round the predictions
        \frac{  2 \times \left | T \right | \cap \left | P \right |}{ \left | T \right | +  \left | P \right |  }
        where T is ground truth mask and P is the prediction mask
        """

        intersection = tf.reduce_sum(target * prediction, axis=axis)
        union = tf.reduce_sum(target + prediction, axis=axis)
        numerator = tf.constant(2.) * intersection + smooth
        denominator = union + smooth
        coef = numerator / denominator

        return tf.reduce_mean(coef)

    def specificity(self, y_true, y_pred):
        """
        Compute specificity.
        """
        # Threshold predictions
        y_pred = K.backend.round(y_pred)

        # Count true negatives
        tn = K.backend.sum(K.backend.round(K.backend.clip((1-y_true) * (1-y_pred), 0, 1)))

        # Count false positives
        fp = K.backend.sum(K.backend.round(K.backend.clip((1-y_true) * y_pred, 0, 1)))

        # Compute specificity
        specificity = tn / (tn + fp + K.backend.epsilon())
        return specificity

metrics = Metrics()
sample = '../Task01_BrainTumour/imagesTr/BRATS_001.nii.gz'

#compute all metrics using the mask as the prediction and the target
target = nib.load(sample).get_fdata()
prediction = nib.load(sample).get_fdata()
target = target[:,:,:,0]
print("target shape: ", target.shape)


print("Precision: ", metrics.precision_coef(target, prediction))
print("Dice: ", metrics.dice_coef(target, prediction))
print("IoU: ", metrics.iou_coef(target, prediction))
print("Soft Dice: ", metrics.soft_dice_coef(target, prediction))
print("Specificity: ", metrics.specificity(target, prediction))



target shape:  (240, 240, 155)


InvalidArgumentError: Incompatible shapes: [240,240,155] vs. [240,240,155,4] [Op:Mul]